#Analysis of Sustainable Building in Israel

The Art of Analyzing Big Data- The Data Scientist’s Toolbox
“Housing Market and Data”

#Author: 312141369

#Environment Setup

In [ ]:
!pip install googletrans==4.0.0rc1
!pip install turicreate
!pip install plotly --upgrade
!pip install xlrd --upgrade
!pip install pyproj
!pip install folium
!pip install cv

!mkdir ./images
!mkdir ./videos
!mkdir ./maps
!mkdir ./datasets


mkdir: cannot create directory ‘./images’: File exists
mkdir: cannot create directory ‘./videos’: File exists
mkdir: cannot create directory ‘./maps’: File exists
mkdir: cannot create directory ‘./datasets’: File exists


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Data Preperation

##Data Loading

In [ ]:
import turicreate as tc
import pandas as pd
import numpy as np

data_path = "./datasets"
# green_buildings_in_israel_path = f"{data_path}/green_building_green_buildings_in_israel.xlsx"
# green_buildings_in_israel_url = "https://www.gov.il/BlobFolder/guide/green_building_in_israel/he/green_building_green_buildings_in_israel.xlsx"
# !wget -O $green_buildings_in_israel_path $green_buildings_in_israel_url

In [ ]:
df_green_building = pd.read_excel('/content/drive/MyDrive/Data_Science/Project/green_building_green_buildings_in_israel.xlsx')
df_green_building

,ID,X,Y,ייעוד,רשות מקומית,רחוב,מספר,גוש,חלקה,יחידות דיור,קומות,מטר מרובע,שנת קבלת התו הירוק,חלק בתקן,ציון סופי,דרגה,גרסת תקן,חדש/ שיפוץ,דירוג אנרגטי
0,1,181160,572790,משרדים,באר שבע,מנחם קרויצר,12,38018,12,לא רלוונטי,8,5865,2008,2-משרדים,69,ירוק,2005.0,שיפוץ,NaN
1,2,195086,717680,מוסד,זכרון יעקב,גני רמת הנדיב,NaN,12762,9,לא רלוונטי,2,2800,2008,2-משרדים,77.6,מצטיין,2005.0,חדש,NaN
2,3,180022,664212,בנק,תל אביב-יפו,הארבעה,19,7101,12,לא רלוונטי,1,551,2009,2-משרדים,NaN,ירוק,2005.0,חדש,NaN
3,4,196654,744037,משרדים,חיפה,אנדריי סחרוב,NaN,NaN,NaN,לא רלוונטי,7,50000,2010,2-משרדים,77.3,מצטיין,2005.0,חדש,NaN
4,5,208960,727704,בנק,יקנעם,שדרות רבין,9,NaN,NaN,לא רלוונטי,NaN,NaN,2010,2-משרדים,65,ירוק,2005.0,חדש,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
444,451,678116,187642,מגורים- רווי,רעננה,יצחק גרציאני,7,7611,14,15,7,2022,2019,2- מגורים,56,1 כוכב,2011.0,חדש,C
445,452,678064,187603,מגורים- רווי,רעננה,יצחק גרציאני,9,7611,14,14,6,1848,2019,2- מגורים,56,1 כוכב,2011.0,חדש,C
446,453,646709,180049,מגורים- רווי,נס ציונה,הכרמים,16,3750,169,12,5,3800,2019,2- מגורים,56,1 כוכב,2011.0,חדש,A
447,454,646688,180075,מגורים- רווי,נס ציונה,הכרמים,18,3750,169,11,5,1400,2019,2- מגורים,56,1 כוכב,2011.0,חדש,A


In [ ]:
df_active_green_building = pd.read_excel('/content/drive/MyDrive/Data_Science/Project/active_green_buildings.xlsx')
df_active_green_building = df_active_green_building[["project_id","X","Y","standard_designation_name", "main_use_name","municipality_name","building_street","building_address_number","gush","helka","residential_units","floors_above_ground","designation_area","standard_name","project_general_last_update","building_info_last_update","designation_info_last_update","certificate_date_pre","building_count","certification_status", "route"]]
df_active_green_building

,project_id,X,Y,standard_designation_name,main_use_name,municipality_name,building_street,building_address_number,gush,helka,residential_units,floors_above_ground,designation_area,standard_name,project_general_last_update,building_info_last_update,designation_info_last_update,certificate_date_pre,building_count,certification_status,route
0,25,174558.000000,641668.000000,מגורים,מגורים,יבנה,המשוט,2022-03-01 00:00:00,5961,"5,6",95,6.0,16000,"2016 סיוון התשע""ו - מאי",2022-01-07 11:31:19,2022-01-04 06:45:58,2022-01-04 06:45:38,NaT,1.0,שלב ב,חדש
1,26,204423.500795,552535.687117,חינוך - בתי ספר,חינוך,דימונה,הרבי מילווביטש,NaN,39861,31,0,1.0,3500,"2016 סיוון התשע""ו - מאי",2022-01-31 11:17:28,2021-11-22 09:07:54,2021-06-27 19:13:12,NaT,1.0,התקשרות,חדש
2,42,181042.871976,666181.479524,חינוך - בתי ספר,חינוך,תל אביב -יפו,קריאל גרדוש,NaN,6108,729,0,3.0,2788,"2016 סיוון התשע""ו - מאי",2021-08-29 09:52:45,2021-06-08 07:17:03,2020-11-10 14:35:30,NaT,1.0,שלב ב,חדש
3,57,NaN,NaN,NaN,חינוך,אשדוד,NaN,NaN,3174,NaN,0,5.0,8000,"2016 סיוון התשע""ו - מאי",2021-11-08 06:54:55,NaT,2020-11-10 14:35:30,NaT,NaN,שלב א,חדש
4,61,180137.574601,667755.889143,מגורים,מגורים,תל אביב -יפו,שי עגנון,28,6634,336,114,12.0,12400,"2016 סיוון התשע""ו - מאי",2022-01-12 17:39:30,2021-12-22 11:49:16,2021-04-14 11:25:11,2019-09-13,1.0,שלב ב,חדש
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
372,754,178233.480284,664732.378584,מגורים,מגורים,תל אביב -יפו,בן יהודה,15,6909,61,14,7.0,0,"2016 סיוון התשע""ו - מאי",2022-01-31 10:31:45,2021-12-14 09:23:42,2021-12-12 13:23:53,2021-12-14,1.0,שלב מקדמי,חדש
373,757,183401.043345,663732.324047,מגורים,מגורים,רמת גן,צאלים,4,6145,1141,73,7.0,5000,"2016 סיוון התשע""ו - מאי",2022-01-13 10:03:18,2021-12-22 09:20:56,2021-12-22 09:20:31,NaT,1.0,התקשרות,חדש
374,773,193062.322510,704381.847926,מגורים,מגורים,חדרה,אחד העם,NaN,10037,278,12,7.0,448,"2016 סיוון התשע""ו - מאי",2022-01-31 12:10:55,2022-01-12 08:38:33,2022-01-12 08:38:19,2021-12-29,1.0,שלב מקדמי,חדש
375,777,188102.141840,677525.315856,התקהלות ציבורית + ג.ת 1,התקהלות ציבורית,רעננה,אלפסי,25,7654,330,0,2.0,530,"2016 סיוון התשע""ו - מאי",2022-01-11 08:23:03,2022-01-08 20:21:31,2022-01-08 20:30:09,2022-01-08,1.0,שלב מקדמי,חדש


In [ ]:
# municipalities_path = f"{data_path}/municipalities.xlsx"
# municipalities_url = "https://www.cbs.gov.il/he/publications/LochutTlushim/2020/p_libud_2019.xlsx"
# !wget -O $municipalities_path $municipalities_url


In [ ]:
municipalities_excel = pd.ExcelFile('/content/drive/MyDrive/Data_Science/Project/municipalities_1.xlsx') 
sheets = municipalities_excel.sheet_names
print(sheets)

['נתונים פיזיים ונתוני אוכלוסייה ', 'נתוני תקציב', 'סיכומים לפי מעמד מוניציפלי', 'סקרי כוח אדם והוצאות משק בית', 'נתוני הסקר החברתי']


In [ ]:
df_municipalities = pd.read_excel('/content/drive/MyDrive/Data_Science/Project/municipalities_1.xlsx', sheet_name=sheets[0])#), header=[0,1])
columns_dict = {}
for col_name in df_municipalities.columns:
  columns_dict[col_name] = col_name.strip().replace("\n", " ")

df_municipalities = df_municipalities.rename(columns=columns_dict)
df_municipalities

,שם הרשות,סמל הרשות,מחוז,מעמד מוניציפלי,"מרחק מגבול מחוז תל אביב (ק""מ)",שנת קבלת מעמד מוניציפלי,"מרחב ימי, רשויות מקומיות הגובלות בחוף הים",מספר חברי מועצה,סמל ועדת תכנון ובנייה,שם ועדת תכנון ובנייה,"שטח (קמ""ר)","צפיפות אוכלוסייה לקמ''ר ביישובים שמנו 5,000 תושבים ויותר","סה""כ אוכלוסייה בסוף השנה",יהודים ואחרים (אחוזים),יהודים (אחוזים מתוך יהודים ואחרים),ערבים (אחוזים),מוסלמים (אחוזים מתוך האוכלוסייה הערבית),נוצרים (אחוזים מתוך האוכלוסייה הערבית),דרוזים (אחוזים מתוך האוכלוסייה הערבית),"סה""כ גברים בסוף השנה","סה""כ נשים בסוף השנה",אחוז באוכלוסייה בסוף השנה | בני 4-0,אחוז באוכלוסייה בסוף השנה | בני 9-5,אחוז באוכלוסייה בסוף השנה | בני 14-10,אחוז באוכלוסייה בסוף השנה | בני 19-15,אחוז באוכלוסייה בסוף השנה | בני 29-20,אחוז באוכלוסייה בסוף השנה | בני 44-30,אחוז באוכלוסייה בסוף השנה | בני 59-45,אחוז באוכלוסייה בסוף השנה | בני 64-60,אחוז באוכלוסייה בסוף השנה | בני 65 ומעלה,אחוז באוכלוסייה בסוף השנה | בני 17-0,אחוז באוכלוסייה בסוף השנה | בני 75 ומעלה,"יחס תלות (ל-1,000 תושבים בלתי תלויים)",לידות חי,שיעור פריון כולל,פטירות תינוקות (ממוצע),"שיעור פטירות תינוקות ל-1,000 לידות חי",פטירות,"שיעור פטירות מתוקנן ל-1,000 תושבים","ריבוי טבעי סה""כ",...,תשתית ותחבורה | אחוז מסך כל שטח שיפוט,"מבנים חקלאיים | שטח בקמ""ר",מבנים חקלאיים | אחוז מסך כל שטח שיפוט,"גינון לנוי, פארק ציבורי | שטח בקמ""ר","גינון לנוי, פארק ציבורי | אחוז מסך כל שטח שיפוט","יער וחורש | שטח בקמ""ר",יער וחורש | אחוז מסך כל שטח שיפוט,"מטעים | שטח בקמ""ר",מטעים | אחוז מסך כל שטח שיפוט,"גידולי שדה | שטח בקמ""ר",גידולי שדה | אחוז מסך כל שטח שיפוט,"שטח פתוח אחר | שטח בקמ""ר",שטח פתוח אחר | אחוז מסך כל שטח שיפוט,אוכלוסייה סוף 2013 סך הכל אלפים,"צפיפות אוכלוסייה לשטח בנוי למגורים | נפשות לקמ""ר","ממוצע המרחקים (ק""מ)","ממוצע המרחקים משוקלל (ק""מ)","סה""כ יישובים במועצה","יישובים עירוניים, יהודיים 10,000-19,999 תושבים","יישובים עירוניים, יהודיים 5,000-9,999 תושבים","יישובים עירוניים, לא-יהודיים 5,000-9,999 תושבים","יישובים עירוניים, יהודיים 2,000-4,999 תושבים","יישובים עירוניים, מושבים 2,000-4,999 תושבים","יישובים עירוניים, מושבים שיתופיים 2,000-4,999 תושבים","יישובים עירוניים, קיבוצים 2,000-4,999 תושבים","יישובים עירוניים, לא-יהודיים 2,000-4,999 תושבים",מושבים (לרבות כפרים שיתופיים),מושבים שיתופיים,קיבוצים,"יישובים מוסדיים, יהודיים","יישובים מוסדיים, לא-יהודיים",יישובים קהילתיים,"יישובים כפריים אחרים, יהודיים","יישובים כפריים אחרים, לא-יהודיים",בחירות מוניציפליות לראש ראשות - סבב א 30/10/18 | אחוזי הצבעה,בחירות מוניציפליות לראש ראשות - סבב ב 13/11/18 | אחוזי הצבעה,בחירות לכנסת ה-21 9/4/19 | אחוזי הצבעה,בחירות לכנסת ה-22 17/9/19 | אחוזי הצבעה,בחירות לכנסת ה-23 02/03/20 | אחוזי הצבעה,בחירות לכנסת ה-24 23/03/21 | אחוזי הצבעה
0,אום אל-פחם,2710,חיפה,עירייה,57.8,1984,0,17,354,עירון,25.956107,2187.4,56777,-,-,99.9,100,-,-,28947,27829,9.799743,9.702873,10.365113,10.972753,18.671293,17.908660,15.171636,2.751114,4.658577,36.159008,1.608046,835,1080.0,2.54,3.2,( 2.8),186,6.12,894.0,...,1.4,0.01,-,-,-,6.31,24.2,5.69,21.8,0.15,0.6,7.24,27.8,50.6,9134,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,74.24,66.818194,45.135369,51.334527,63.103806,33.147473
1,אופקים,31,הדרום,עירייה,86.8,1995,0,15,601,אופקים,16.352374,1990.8,32555,99.3,93.4,-,-,-,-,16107,16448,13.967133,10.873906,8.143142,7.061895,16.040547,18.553218,12.686223,3.937951,8.732914,36.937490,3.357395,952,942.0,3.97,3.4,( 4.0),180,6.18,762.0,...,1,-,-,0.09,0.9,0.11,1.1,-,-,1.99,19.4,4.55,44.3,25.1,9843,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,68.88,-,65.499785,66.482978,67.193948,60.958874
2,אור יהודה,2400,תל אביב,עירייה,0,1988,0,15,510,אור יהודה,6.66364,5518,36770,99.8,97.7,-,-,-,-,18227,18543,8.724504,8.126190,7.136252,7.133533,13.815611,20.146859,16.154474,5.303236,13.459342,28.264890,4.979603,804,631.0,2.51,1.2,( 1.8),254,5.03,377.0,...,4.9,0.01,0.1,0.04,0.6,-,-,0.35,5.2,2.03,30.3,1.04,15.7,35.5,16825,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,65.75,-,69.558284,69.944060,70.079792,66.773592
3,אור עקיבא,1020,חיפה,עירייה,39.5,2001,0,13,353,יישובי הברון,5.546007,3506.6,19447,99.5,91.7,-,-,-,-,9573,9874,9.142798,8.464030,6.947087,6.155191,12.603486,20.68699

##Preprocess

In [ ]:
#TODO: Execlode 2022 

###Translator

In [ ]:
from googletrans import Translator
#initialize the Translator
translator = Translator()

source_lan = "he"
translated_to= "en" 

#translate text
# text = "מגורים"
# translated_text = translator.translate(text, src=source_lan, dest = translated_to).text

### Coordinates Convertion

In [ ]:
import pyproj # Import the pyproj module 
  
# Define a projection with Proj4 notation, in this case an Israel Transverse Mercator (ITM) coordinates
itm=pyproj.CRS("+proj=tmerc +lat_0=31.73439361111111 +lon_0=35.20451694444445 +k=1.0000067 +x_0=219529.584 +y_0=626907.39 +ellps=GRS80 +towgs84=-48,55,52,0,0,0,0 +units=m +no_defs") 
# Define common projection
wgs84=pyproj.CRS("EPSG:4326")

from functools import lru_cache
import time
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="Data Science Education App") #  Using OpenStreetMap Nominatim

@lru_cache(maxsize=256)
def get_location(loc):
    time.sleep(1)
    return geolocator.geocode(loc)

def loc_coordinates(location_names):
  locations_coordinates = {}
  for location_name in location_names:
    location = get_location(location_name+', Israel')
    locations_coordinates[location_name]={}
    locations_coordinates[location_name]["longitude"] = location.longitude  
    locations_coordinates[location_name]["latitude"] = location.latitude  
  return locations_coordinates

israel_border_min_lat = 29.48
israel_border_max_lat = 33.33
israel_border_min_long = 34.26
israel_border_max_long = 35.89

def coordinate_within_israel(lat, long):
  return israel_border_min_lat< lat < israel_border_max_lat and israel_border_min_long< long < israel_border_max_long
  

###Green Buildings

In [ ]:
#Translating columns
df_green_building_column_names = {
    "ID": "Id",
    "X": "X",
    "Y": "Y",
    "ייעוד": "Designation",
    "רשות מקומית": "Local Authority",
    "רחוב": "Street",
    "מספר": "Building Number",
    "גוש": "Bloc",
    "חלקה": "Plot",
    "יחידות דיור": "Housing Units",
    "קומות": "Floors",
    "מטר מרובע": "Square Meter",
    "שנת קבלת התו הירוק": "Year",
    "חלק בתקן": "Part of Standard",
    "ציון סופי": "Final Score",
    "דרגה": "Rank",
    "גרסת תקן": "Standard Version",
    "חדש/ שיפוץ": "New/Renovation",
    "דירוג אנרגטי": "Energetic Rating"
}
df_green_building = df_green_building.rename(columns=df_green_building_column_names)

Missing values summary

In [ ]:
df_green_building["X"] = df_green_building["X"].apply(lambda r: r if r != 'חסר' else None)
df_green_building["Y"] = df_green_building["Y"].apply(lambda r: r if r != 'חסר' else None)
df_green_building.isnull().sum(axis=0)

Id                    0
X                    12
Y                    12
Designation           0
Local Authority       0
Street                3
Building Number      18
Bloc                 59
Plot                 86
Housing Units         1
Floors               97
Square Meter        152
Year                  0
Part of Standard      0
Final Score           6
Rank                  5
Standard Version     24
New/Renovation        3
Energetic Rating    230
dtype: int64

In [ ]:
def convert_to_float(value):
  if type(value) is str and value!= 'חסר':
    return float(value.replace(',','.'))
  return value
df_green_building["Square Meter"] = df_green_building["Square Meter"].apply(lambda r: convert_to_float(r)) 

In [ ]:
na_fill_floors = df_green_building["Floors"].apply(lambda r: r if r != 'חסר' else None).dropna().median()
na_fill_squere_meter = df_green_building["Square Meter"].apply(lambda r: r if r != 'חסר' else None).dropna().median()

In [ ]:
df_green_building["Floors"] = df_green_building["Floors"].fillna(na_fill_floors).apply(lambda r: r if r != 'חסר' else na_fill_floors) 
df_green_building["Housing Units"] = df_green_building["Housing Units"].fillna(0)
df_green_building["Square Meter"] = df_green_building["Square Meter"].fillna(na_fill_squere_meter).apply(lambda r: r if r != 'חסר' else na_fill_squere_meter) #Many rows missing
df_green_building["New/Renovation"] = df_green_building["New/Renovation"].apply(lambda r: str(r).strip())

In [ ]:
municipalitites = {
    "באר יעקוב": "באר יעקב",
    "הרצלייה": "הרצליה",
    "טירת כרמל": "טירת הכרמל",
    "יוקנעם עילית": "יקנעם עילית",
    "יקנעם": "יקנעם עילית",
    'מ"א גזר': "גזר",
    'מ"א משגב': "משגב",
    'מ"א תמר': "תמר",
    "קריית גת": "קרית גת",
    "קריית אונו": "קרית אונו",
    "תל אביב -יפו": "תל אביב-יפו",
    'ל"ר- (רשות שדות התעופה בישראל)': "רשות שדות התעופה",
    "כרכור": "פרדס חנה-כרכור",
    "נהריה": "נהרייה",
    "	רמת רזיאל": "מטה יהודה",
    "רמת רזיאל": "מטה יהודה",
    "צור יצחק": "דרום השרון"
}
def clean_local_auth(local_auth):
  local_auth = local_auth.strip()
  if local_auth in municipalitites:
    local_auth = municipalitites[local_auth]
  return local_auth

df_green_building["Local Authority"] = df_green_building["Local Authority"].apply(lambda r: clean_local_auth(r))

Missing Address

We are only intrested in removing samples that has no address information. Either having full coordinates or a street address are sufficient. 

In [ ]:
df_green_building = df_green_building.apply(lambda r: r if (not pd.isnull(r["X"]) and not pd.isnull(r["Y"])) or (not pd.isnull(r["Local Authority"]) and not pd.isnull(r["Street"])) else None, axis=1)

Values Converstion and Translation

In [ ]:
# df_green_building["Year"] = pd.to_datetime(df_green_building["Year"], format='%Y').dt.date

In [ ]:
# for text in df_green_building["Designation"].unique():
#   print(f"{text}: {translator.translate(text, src=source_lan, dest=translated_to,).text}")

designation_convertion_specific = {
    "משרדים": "Offices",
    "מוסד": "Institution",
    "בנק": "Bank",
    "מגורים-רווי": "Residence",
    "מגורים-פרטי": "Residence",
    "בית משפט": "Court",
    "גן/מעון": "Kindergarten",
    'בי"ס': "School",
    "אחר": "Other",
    "בית חולים/מרפאה": "Health care facility",
    "השכלה גבוהה": "Higher education",
    "בית ספר": "School",
    "משרדים-מסחר": "Commerce",
    "אזור תעשיה": "Industrial",
    "בית חולים": "Health care facility",
    "משרדים-רגיל": "Offices",
    "משרדים-גרעין ומעטפת": "Offices",
    "מגורים- רווי": "Residence",
    "מלון": "Tourism",
    "מבנה ציבור- התקהלות": "Public structure"
}

designation_convertion = {
    "Offices": "Offices",
    "Institution": "Institution",
    "Bank": "Institution",
    "Residence": "Residence",
    "Court": "Institution",
    "Kindergarten": "Education",
    'School': "Education",
    "Other": "Other",
    "Health care facility": "Health care facility",
    "Higher education": "Education",
    "Commerce": "Commerce",
    "Industrial": "Industrial",
    "Tourism": "Tourism",
    "Public structure": "Public structure"
}
df_green_building["Designation Specific"] = df_green_building["Designation"].apply(lambda r: designation_convertion_specific[r])
df_green_building["Designation"] = df_green_building["Designation Specific"].apply(lambda r: designation_convertion[r])


In [ ]:
# df_green_building["Housing Units"].unique()
df_green_building["Housing Units"] = df_green_building["Housing Units"].apply(lambda r: r if type(r) == int else 0) #TODO: irrelevant

Coordinates Convertion

Convert coordinates for the samples that has ITM x,y Coordinates

In [ ]:
long_lat = pyproj.transform(itm, wgs84, df_green_building["X"].tolist(), df_green_building["Y"].tolist())
df_green_building["lat"] = long_lat[0]
df_green_building["long"] = long_lat[1]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  """Entry point for launching an IPython kernel.


Several x,y coordinates are wrong (outside Israel). For these, coordinates will be extrcted from the local authority field. 

In [ ]:
df_wrong_coor_smaller_min_lat = df_green_building[df_green_building["lat"]<israel_border_min_lat]
df_wrong_coor_smaller_min_long = df_green_building[df_green_building["long"]<israel_border_min_long]
df_wrong_coor_smaller_max_lat = df_green_building[df_green_building["lat"]>israel_border_max_lat]
df_wrong_coor_smaller_max_long = df_green_building[df_green_building["long"]>israel_border_max_long]
df_wrong_coor = pd.merge(df_wrong_coor_smaller_min_lat, df_wrong_coor_smaller_min_long, how="left")
df_wrong_coor = pd.merge(df_wrong_coor,df_wrong_coor_smaller_max_lat, how="left")
df_wrong_coor = pd.merge(df_wrong_coor_smaller_max_long,df_wrong_coor, how="left")

find coordinates for the samples with no ITM x,y coordinates

In [ ]:
missing_coor_locations = {}
for local_auth in df_green_building.loc[df_green_building[df_green_building["X"].isna()]["Y"].isna().index]["Local Authority"].unique():
  missing_coor_locations[local_auth] = translator.translate(local_auth, src=source_lan, dest=translated_to,).text

for local_auth in df_wrong_coor["Local Authority"].unique():
  if local_auth not in missing_coor_locations:
    missing_coor_locations[local_auth] = translator.translate(local_auth, src=source_lan, dest=translated_to,).text
#do the same for addresses outside Israel

loc_cords = loc_coordinates(missing_coor_locations.values())
print("loc_cords", loc_cords)
print("missing_coor_locations", missing_coor_locations)

loc_cords {'Kfar Saba': {'longitude': 34.907459, 'latitude': 32.1773471}, 'Ashkelon': {'longitude': 34.5730157, 'latitude': 31.6644874}, 'Or Akiva': {'longitude': 34.9195677, 'latitude': 32.5090151}, "Be'er Ya'akov": {'longitude': 34.8398648, 'latitude': 31.9443532}, 'Kfar Yona': {'longitude': 34.9321169, 'latitude': 32.314504}, 'Netanya': {'longitude': 34.8566246, 'latitude': 32.3286181}, 'Tel Aviv Jaffa': {'longitude': 34.7818064, 'latitude': 32.0852997}, 'Rishon Lezion': {'longitude': 34.8101149, 'latitude': 31.9635712}, 'Petah Tikva': {'longitude': 34.8859985, 'latitude': 32.0877639}, 'Beer Sheva': {'longitude': 34.7925181, 'latitude': 31.2457442}, 'Herzliya': {'longitude': 34.83082794429146, 'latitude': 32.1676842}, "Ra'anana": {'longitude': 34.8675905, 'latitude': 32.1861481}, 'Ness Ziona': {'longitude': 34.7990609, 'latitude': 31.9295577}}
missing_coor_locations {'כפר סבא': 'Kfar Saba', 'אשקלון': 'Ashkelon', 'אור עקיבא': 'Or Akiva', 'באר יעקב': "Be'er Ya'akov", 'כפר יונה': 'Kfar

In [ ]:
def fill_coordinates(row):
  if not pd.isnull(row["X"]) and coordinate_within_israel(row["lat"],row["long"]):
    return row
  # if (row["X"] != np.nan and row["Y"] != np.nan):
  #   return row
  else:
    row["lat"] = loc_cords[missing_coor_locations[row["Local Authority"]]]["latitude"]
    row["long"] = loc_cords[missing_coor_locations[row["Local Authority"]]]["longitude"]
    return row
    
df_green_building = df_green_building.apply(lambda r: fill_coordinates(r), axis=1)


###Active Green Buildings

In [ ]:
df_active_green_building_column_names = {
  'project_id': 'Id',
  'municipality_name': 'Local Authority', 
  'building_street': 'Street',
  'building_address_number': 'Building Number',
  'gush': 'Bloc',
  'helka': 'Plot',
  'residential_units': 'Housing Units',
  'floors_above_ground': 'Floors',
  'designation_area': 'Square Meter',
  'main_use_name': 'Designation',
  'standard_name': 'Standard Version',
  'route': 'New/Renovation'
}
df_active_green_building = df_active_green_building.rename(columns=df_active_green_building_column_names)

Missing values summary

In [ ]:
df_active_green_building.isnull().sum(axis=0)

Id                                0
X                                17
Y                                17
standard_designation_name        17
Designation                       0
Local Authority                   0
Street                           25
Building Number                  37
Bloc                              0
Plot                             17
Housing Units                     0
Floors                            0
Square Meter                      0
Standard Version                  0
project_general_last_update       0
building_info_last_update        17
designation_info_last_update      0
certificate_date_pre            102
building_count                   17
certification_status              0
New/Renovation                    0
dtype: int64

In [ ]:
df_active_green_building["Local Authority"] = df_active_green_building["Local Authority"].apply(lambda r: clean_local_auth(r))

Missing Address

We are only intrested in removing samples that has no address information. Either having full coordinates or a local authority are sufficient. 

In [ ]:
df_active_green_building = df_active_green_building.apply(lambda r: r if (not pd.isnull(r["X"]) and not pd.isnull(r["Y"])) or (not pd.isnull(r["Local Authority"])) else None, axis=1)

Values Converstion and Translation

In [ ]:
# for text in df_active_green_building["main_use_name"].unique():
#   if not text:
#     continue
#   print(f"{text}: {translator.translate(text, src=source_lan, dest=translated_to,).text}")

In [ ]:
designation_convertion = {
  'חינוך': 'Education',
  'מגורים': 'Residence',
  'מסחר': 'Commerce',
  'התקהלות ציבורית': 'Public structure',
  'תעשייה': 'Industrial',
  'משרדים': 'Offices',
  'מוסדות בריאות': 'Health care facility',
  'אכסון תיירותי' : 'Tourism',
  'גרעין ומעטפת - משרדים': 'Offices'
}

# def convert_designation(row):
#   if row["Designation"]:
#     return row
#   else:
#     print(f"row['Designation']: {row['Designation']}, row['standard_designation_name']: {row['standard_designation_name']}")
#     row["Designation"] = row["standard_designation_name"]
#     return row
#    
# df_active_green_building = df_active_green_building.apply(lambda r: convert_designation(r), axis=1)
df_active_green_building["Designation"] = df_active_green_building["Designation"].apply(lambda r: designation_convertion[r])

In [ ]:
def update_earliest_date(row):
  if not pd.isna(row["certificate_date_pre"]):
    row["Earliest Date"] = row["certificate_date_pre"]
    return row
  else:
    row["Earliest Date"] = min(row["designation_info_last_update"], row["building_info_last_update"], row["project_general_last_update"])
    return row

df_active_green_building = df_active_green_building.apply(lambda r: update_earliest_date(r), axis=1)
df_active_green_building = df_active_green_building.drop(["designation_info_last_update",'certificate_date_pre', 'building_info_last_update', 'project_general_last_update'], 1)
# Extract Earliest Year from Earliest Date
df_active_green_building['Year'] = pd.DatetimeIndex(df_active_green_building['Earliest Date']).year


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  # Remove the CWD from sys.path while we load stuff.


Coordinates Convertion

Convert coordinates for the samples that has ITM x,y Coordinates

In [ ]:
long_lat = pyproj.transform(itm, wgs84, df_active_green_building["X"].tolist(), df_active_green_building["Y"].tolist())
df_active_green_building["lat"] = long_lat[0]
df_active_green_building["long"] = long_lat[1]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  """Entry point for launching an IPython kernel.


find coordinates for the samples with no ITM x,y coordinates

In [ ]:
missing_coor_locations = {}
for local_auth in df_active_green_building.loc[df_active_green_building[df_active_green_building["X"].isna()]["Y"].isna().index]["Local Authority"].unique():
  missing_coor_locations[local_auth] = translator.translate(local_auth, src=source_lan, dest=translated_to,).text

loc_cords = loc_coordinates(missing_coor_locations.values())
print("loc_cords", loc_cords)
print("missing_coor_locations", missing_coor_locations)

loc_cords {'Ashdod': {'longitude': 34.6529922, 'latitude': 31.7977314}, "Ra'anana": {'longitude': 34.8675905, 'latitude': 32.1861481}, 'Bat Yam': {'longitude': 34.7505283, 'latitude': 32.0154565}, 'Holon': {'longitude': 34.78581210180328, 'latitude': 32.015910399999996}, 'Netanya': {'longitude': 34.8566246, 'latitude': 32.3286181}, 'Herzliya': {'longitude': 34.83082794429146, 'latitude': 32.1676842}, 'Ness Ziona': {'longitude': 34.7990609, 'latitude': 31.9295577}, 'Kfar Yona': {'longitude': 34.9321169, 'latitude': 32.314504}, 'Ramat Hasharon': {'longitude': 34.83019949563153, 'latitude': 32.134855200000004}, 'Rishon Lezion': {'longitude': 34.8101149, 'latitude': 31.9635712}, 'Umm al-Fahm': {'longitude': 35.152491, 'latitude': 32.5158385}, 'Tel Aviv Jaffa': {'longitude': 34.7818064, 'latitude': 32.0852997}}
missing_coor_locations {'אשדוד': 'Ashdod', 'רעננה': "Ra'anana", 'בת ים': 'Bat Yam', 'חולון': 'Holon', 'נתניה': 'Netanya', 'הרצליה': 'Herzliya', 'נס ציונה': 'Ness Ziona', 'כפר יונה': 

In [ ]:
df_active_green_building = df_active_green_building.apply(lambda r: fill_coordinates(r), axis=1)


###Municipalities Data

In [ ]:
# print(df_municipalities.columns.tolist())
df_municipalities = df_municipalities[["שם הרשות", 'מרחק מגבול מחוז תל אביב (ק"מ)', 'שטח  (קמ"ר)', 'סה"כ  אוכלוסייה בסוף השנה', 'דמי אבטלה ממוצעים ליום (ש"ח)', 'שכר ממוצע לחודש של שכירים (ש"ח) | כלל השכירים', 'מספר השכירים', 'אחוז השכירים המשתכרים עד שכר מינימום',  'מספר העצמאים', 'הכנסה ממוצעת לחודש של העצמאים (ש"ח)', 'אחוז העצמאים המשתכרים עד מחצית השכר הממוצע', "מדד אי-השוויון שכירים (מדד ג'יני, 0 שוויון מלא)", 'אחוז תלמידים נושרים | סה"כ', 'אחוז זכאים לתעודת בגרות מבין תלמידי כיתות יב', "השכלה גבוהה | 'אחוז סטודנטים מתוך כלל האוכלוסייה", "צפיפות אוכלוסייה לקמ''ר ביישובים שמנו 5,000 תושבים ויותר"]]
df_municipalities["שם הרשות"] = df_municipalities["שם הרשות"].apply(lambda r: clean_local_auth(r))
df_municipalities_columns_names = {
    'שם הרשות': 'Local Authority',
    'מרחק מגבול מחוז תל אביב (ק"מ)': 'Distance from Tel-Aviv District Border Line (km)',
    'שטח  (קמ"ר)': 'Area (square km)',
    'סה"כ  אוכלוסייה בסוף השנה': 'Population',
    'דמי אבטלה ממוצעים ליום (ש"ח)': 'Avrege Daily Unemployment Benefit',
    'שכר ממוצע לחודש של שכירים (ש"ח) | כלל השכירים': 'Average Monthly Wage of Employees',
    'מספר השכירים': 'Number of Employees',
    'אחוז השכירים המשתכרים עד שכר מינימום': 'Percentage of Employees Earning Up To Minimum Wage',
    'מספר העצמאים': 'Number of Self-Employed',
    'הכנסה ממוצעת לחודש של העצמאים (ש"ח)': 'Average Monthly Wage of Self-Employed',
    'אחוז העצמאים המשתכרים עד מחצית השכר הממוצע': 'Percentage of Self-Employed Earning Up To Half the Average Wage',
    "מדד אי-השוויון שכירים (מדד ג'יני, 0 שוויון מלא)": 'Employees Inequality Index (Gini Index, 0 is complete equality)',
    'אחוז תלמידים נושרים | סה"כ': 'Percentage of students dropping out',
    "אחוז זכאים לתעודת בגרות מבין תלמידי כיתות יב": 'Percentage of matriculation certificate eligible among 12th graders',
    "השכלה גבוהה | 'אחוז סטודנטים מתוך כלל האוכלוסייה": 'Percentage of students out of the general population',
    "צפיפות אוכלוסייה לקמ''ר ביישובים שמנו 5,000 תושבים ויותר": 'Population density per square kilometer',
}
df_municipalities = df_municipalities.rename(columns=df_municipalities_columns_names)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
def fill_missing_with_median(df, col_name, missing_str):
  na_fill = df[col_name].apply(lambda r: r if r != missing_str else None).dropna().median()
  df[col_name] = df[col_name].fillna(na_fill).apply(lambda r: r if r != missing_str else na_fill) 
  return df

In [ ]:
df_municipalities = fill_missing_with_median(df_municipalities, col_name="Distance from Tel-Aviv District Border Line (km)", missing_str='-')
df_municipalities = fill_missing_with_median(df_municipalities, col_name="Area (square km)", missing_str='..')
df_municipalities = fill_missing_with_median(df_municipalities, col_name="Percentage of students dropping out", missing_str='..')
df_municipalities = fill_missing_with_median(df_municipalities, col_name="Percentage of matriculation certificate eligible among 12th graders", missing_str='..')
df_municipalities = fill_missing_with_median(df_municipalities, col_name="Population density per square kilometer", missing_str='..')


##Merging

In [ ]:
df_green_building_temp = df_green_building.drop(['X','Y', 'Building Number', 'Bloc', 'Plot','Energetic Rating','Designation Specific','Rank','Final Score','Part of Standard','Standard Version','New/Renovation'], 1)
df_active_green_building_temp = df_active_green_building.drop(['X','Y', 'standard_designation_name','Earliest Date', 'Building Number', 'Bloc', 'Plot','Standard Version', 'building_count','certification_status'], 1)
df_combined_green_building = pd.concat([df_green_building_temp,df_active_green_building_temp])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


In [ ]:
df_combined_green_building["Year Date"] = pd.to_datetime(df_combined_green_building["Year"], format='%Y').dt.date

In [ ]:
df_combined_green_building

,Id,Designation,Local Authority,Street,Housing Units,Floors,Square Meter,Year,lat,long,New/Renovation,Year Date
0,1,Offices,באר שבע,מנחם קרויצר,0,8.0,5865.0,2008,31.246124,34.802489,NaN,2008-01-01
1,2,Institution,זכרון יעקב,גני רמת הנדיב,0,2.0,2800.0,2008,32.553139,34.945037,NaN,2008-01-01
2,3,Institution,תל אביב-יפו,הארבעה,0,1.0,551.0,2009,32.070562,34.786871,NaN,2009-01-01
3,4,Offices,חיפה,אנדריי סחרוב,0,7.0,50000.0,2010,32.790836,34.961088,NaN,2010-01-01
4,5,Institution,יקנעם עילית,שדרות רבין,0,8.0,3961.0,2010,32.643746,35.092642,NaN,2010-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...
372,754,Residence,תל אביב-יפו,בן יהודה,14,7.0,0.0,2021,32.075191,34.767907,חדש,2021-01-01
373,757,Residence,רמת גן,צאלים,73,7.0,5000.0,2021,32.066350,34.822675,חדש,2021-01-01
374,773,Residence,חדרה,אחד העם,12,7.0,448.0,2021,32.433180,34.923864,חדש,2021-01-01
375,777,Public structure,רעננה,אלפסי,0,2.0,530.0,2022,32.190872,34.872009,חדש,2022-01-01


#Analysis

##HeatMap of green building in Israel
Weighted by the area.

In [ ]:
import folium
from folium.plugins import HeatMap, HeatMapWithTime
tiles = 'OpenStreetMap'
location_lat = df_combined_green_building["lat"].mean()
location_long = df_combined_green_building["long"].mean()
m = folium.Map(location=[location_lat,location_long], zoom_start=8,
               tiles = tiles)


data = [(r[1]['lat'],r[1]['long'],r[1]["Square Meter"]) for r in df_combined_green_building.iterrows()]
HeatMap(data, radius = 11).add_to(m)
m
#TODO:maybe add pin points

##Density Heatmap

In [ ]:
import plotly.express as px


RuntimeError: ignored

RuntimeError: ignored

RuntimeError: ignored

In [ ]:
fig = px.density_heatmap(df_combined_green_building, x="Square Meter", y="Housing Units", 
                         marginal_x="histogram", marginal_y="histogram", range_color=(0, 50),width=500, height=500)
fig.show()

In [ ]:
fig = px.density_heatmap(df_combined_green_building, x="Square Meter", y="Floors",
                         marginal_x="histogram", marginal_y="histogram", range_color=(0, 50),width=500, height=500)
fig.show()

In [ ]:
fig = px.density_heatmap(df_combined_green_building, x="Housing Units", y="Floors",
                         marginal_x="histogram", marginal_y="histogram", range_color=(0, 50),width=500, height=500)
fig.show()

In [ ]:
fig = px.density_heatmap(df_combined_green_building, x="Year", y="Square Meter",
                         marginal_x="histogram", marginal_y="histogram", range_color=(0, 100),width=500, height=500)
fig.show()

In [ ]:
fig = px.scatter_3d(df_combined_green_building, x='Square Meter', y='Floors', z='Housing Units',
              color='Year', opacity=0.9)#, size=[4 for _ in range(len(df_combined_green_building))], )
fig.show()

In [ ]:
fig = px.density_heatmap(df_combined_green_building, x="Designation", y="Square Meter", 
                         marginal_x="histogram", marginal_y="histogram", range_color=(0, 50))
fig.show()

In [ ]:
fig = px.density_heatmap(df_combined_green_building, x="Designation", y="Floors", 
                         marginal_x="histogram", marginal_y="histogram", range_color=(0, 50))
fig.show()

In [ ]:
fig = px.histogram(df_combined_green_building, x="Designation",width=600, height=300)
fig.show()

##Statistics

In [ ]:
def get_summary(df, column_names):
  summary_dict = {}
  for col in column_names:
    des = df[col].describe()
    summary_dict[col] = {
      "Mean": round(des['mean'],3),
      "Stdev": round(des['std'],3),
      "Median": round(des['50%'],3),
      "Min": round(des['min'],3),
      "Max": round(des['max'],3)
    }
  return pd.DataFrame(summary_dict).T

In [ ]:
columns = ['Housing Units', 'Floors', 'Square Meter']
summary_df = get_summary(df_combined_green_building, columns)
print("Statistics")
summary_df

Statistics


,Mean,Stdev,Median,Min,Max
Housing Units,33.082,46.063,25.0,0.0,700.0
Floors,9.169,6.143,8.0,0.0,56.0
Square Meter,6605.826,9713.662,3961.0,0.0,107668.0


In [ ]:
df_combined_green_building['Designation'].describe()

count           826
unique           10
top       Residence
freq            679
Name: Designation, dtype: object

##By Year

In [ ]:
df_green_building_value_counts = df_combined_green_building["Year"].value_counts().sort_index()
df_green_building_value_counts = df_green_building_value_counts.to_frame(name="Count")

years_group_housing = df_combined_green_building[["Year","Housing Units"]].groupby("Year")
years_group_housing_sum = years_group_housing.sum()

years_group_floors = df_combined_green_building[["Year", "Floors"]].groupby("Year")
years_group_floors_sum = years_group_floors.sum()

years_group_area = df_combined_green_building[["Year", "Square Meter"]].groupby("Year")
years_group_area_sum = years_group_area.sum()

df_sum_years = pd.concat([df_green_building_value_counts,years_group_floors_sum, years_group_housing_sum,years_group_area_sum], axis=1, join="outer")

df_sum_years["Housing Units Per Project"] = df_sum_years["Housing Units"]/df_sum_years["Count"]
df_sum_years["Floors Per Project"] = df_sum_years["Floors"]/df_sum_years["Count"]
df_sum_years["Square Meter Per Project"] = df_sum_years["Square Meter"]/df_sum_years["Count"]
df_sum_years = df_sum_years[["Count","Housing Units","Housing Units Per Project","Floors","Floors Per Project", "Square Meter", "Square Meter Per Project"]]
df_sum_years

,Count,Housing Units,Housing Units Per Project,Floors,Floors Per Project,Square Meter,Square Meter Per Project
2008,2,0,0.000000,10.0,5.000000,8665.00,4332.500000
2009,1,0,0.000000,1.0,1.000000,551.00,551.000000
2010,4,0,0.000000,26.0,6.500000,59022.00,14755.500000
2011,3,124,41.333333,33.0,11.000000,16882.00,5627.333333
2012,29,988,34.068966,292.0,10.068966,213049.00,7346.517241
2013,75,2138,28.506667,745.0,9.933333,487693.50,6502.580000
2014,154,4478,29.077922,1354.0,8.792208,784048.00,5091.220779
2015,93,2457,26.419355,736.0,7.913978,724271.00,7787.860215
2016,79,2627,33.253165,662.0,8.379747,470365.15,5953.989241
2017,93,4046,43.505376,1133.0,12.182796,811155.00,8722.096774


###Statistics

In [ ]:
columns = ['Count', 'Housing Units', 'Housing Units Per Project', 'Floors',
       'Floors Per Project', 'Square Meter', 'Square Meter Per Project']
summary_df = get_summary(df_sum_years, columns)
print("Yearly Statistics")
summary_df

Yearly Statistics


,Mean,Stdev,Median,Min,Max
Count,55.067,51.047,38.000,1.0,154.000
Housing Units,1821.733,1596.777,2138.000,0.0,4478.000
Housing Units Per Project,28.207,16.818,29.078,0.0,57.158
Floors,504.900,458.333,434.000,1.0,1354.000
Floors Per Project,8.203,2.945,8.380,1.0,12.183
Square Meter,363760.843,306712.340,299997.000,551.0,811155.000
Square Meter Per Project,6871.577,3001.513,7346.517,551.0,14755.500


###Yearly Changes Visualiztion

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=4, cols=2, subplot_titles=["Project Count","Total Square Meter", "Square Meter Per Project","Total Floors", "Floors Per Project","Total Housing Units", "Housing Units Per Project"],
                    specs=[[{"type": "bar", "colspan": 2}, None], 
                           [{"type": "bar"},{"type": "bar"}],
                           [{"type": "bar"},{"type": "bar"}],
                           [{"type": "bar"},{"type": "bar"}]],
                    column_widths=[0.5, 0.5],vertical_spacing=0.05, horizontal_spacing=0.05)

fig.add_trace(
    go.Bar(x=df_sum_years.index, y=list(df_sum_years["Count"])),
    row=1, col=1)

fig.add_trace(
    go.Bar(x=df_sum_years.index, y=list(df_sum_years["Square Meter"])),
    row=2, col=1)

fig.add_trace(
    go.Bar(x=df_sum_years.index, y=list(df_sum_years["Square Meter Per Project"])),
    row=2, col=2)

fig.add_trace(
    go.Bar(x=df_sum_years.index, y=list(df_sum_years["Floors"])),
    row=3, col=1)

fig.add_trace(
    go.Bar(x=df_sum_years.index, y=list(df_sum_years["Floors Per Project"])),
    row=3, col=2)

fig.add_trace(
    go.Bar(x=df_sum_years.index, y=list(df_sum_years["Housing Units"])),
    row=4, col=1)

fig.add_trace(
    go.Bar(x=df_sum_years.index, y=list(df_sum_years["Housing Units Per Project"])),
    row=4, col=2)

fig.update_layout(height=1000, width=1000, title_text="Changes by Year", showlegend=False)
fig.show()

###HeatMap of over time

In [ ]:
# from collections import defaultdict, OrderedDict

# data = defaultdict(list)
# for _,r in df_combined_green_building.iterrows():
#   data[r["Year Date"]].append([r['lat'], r['long']])
    
# data = OrderedDict(sorted(data.items(), key=lambda t: t[0]))


# tiles = 'OpenStreetMap'
# location_lat = df_combined_green_building["lat"].mean()
# location_long = df_combined_green_building["long"].mean()
# m = folium.Map(location=[location_lat,location_long], zoom_start=8,
#                tiles = tiles)
# # data = [(r[1]['lat'],r[1]['long'],r[1]["Square Meter"]) for r in df_combined_green_building.iterrows()]
# # index = list(df_combined_green_building["Year"])
# hm = HeatMapWithTime(data=list(data.values()), index=list(data.keys()), radius = 11, auto_play=True)#, zindex=20)
# hm.add_to(m)
# folium.LayerControl(collapsed=False).add_to(m)
# m

In [ ]:
import datetime

for year in sorted(df_combined_green_building["Year"].unique()):
  df_cumulative = df_combined_green_building[df_combined_green_building["Year"] <= year]
  m = folium.Map(location=[location_lat,location_long], zoom_start=8,
               tiles = tiles)

  data = [(r[1]['lat'],r[1]['long'],r[1]["Square Meter"]) for r in df_cumulative.iterrows()]
  HeatMap(data, radius = 12).add_to(m)
  m.save(f'./maps/heatmap_over_time{year}.html')

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import os

image_folder = '/content/drive/MyDrive/Data_Science/Project/maps/images'
video_name = './videos/heatmap_over_time.avi'

images = sorted([img for img in os.listdir(image_folder) if img.endswith(".png")])
frame = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = frame.shape

fourcc = cv2.VideoWriter_fourcc('M','J','P','G')
video = cv2.VideoWriter(video_name, fourcc, 1, (width,height))

for image in images:
  # cv.WriteFrame(video, cv2.imread(os.path.join(image_folder, image)))
  frame = cv2.imread(os.path.join(image_folder, image))
  # cv2_imshow(frame)
  video.write(frame)

video.release()
cv2.destroyAllWindows()



In [ ]:
from moviepy.editor import *

clip=VideoFileClip(video_name)
clip.ipython_display(width=500)

 94%|█████████▍| 15/16 [00:00<00:00, 168.96it/s]


###Trendlines

In [ ]:
#dropping 2022 as it is a partial year
df_sum_years_no_2022 = df_sum_years.drop(index=2022)
df_sum_years_no_2022

,Count,Housing Units,Housing Units Per Project,Floors,Floors Per Project,Square Meter,Square Meter Per Project
2008,2,0,0.000000,10.0,5.000000,8665.00,4332.500000
2009,1,0,0.000000,1.0,1.000000,551.00,551.000000
2010,4,0,0.000000,26.0,6.500000,59022.00,14755.500000
2011,3,124,41.333333,33.0,11.000000,16882.00,5627.333333
2012,29,988,34.068966,292.0,10.068966,213049.00,7346.517241
2013,75,2138,28.506667,745.0,9.933333,487693.50,6502.580000
2014,154,4478,29.077922,1354.0,8.792208,784048.00,5091.220779
2015,93,2457,26.419355,736.0,7.913978,724271.00,7787.860215
2016,79,2627,33.253165,662.0,8.379747,470365.15,5953.989241
2017,93,4046,43.505376,1133.0,12.182796,811155.00,8722.096774


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import statsmodels.api as sm

def eval_linear_reg(X,y):
  reg = LinearRegression()
  reg.fit(X, y)
  print("-----------------------------------------------")
  print("The linear model is: Y = {:.5} + {:.5}X".format(reg.intercept_[0], reg.coef_[0][0]))
  print("-----------------------------------------------")
  X2 = sm.add_constant(X)
  est = sm.OLS(y, X2)
  est2 = est.fit()
  print(est2.summary())
  print("pvalues", est2.pvalues)

####Counts

In [ ]:
fig1 = px.scatter(x=df_sum_years.index, y=list(df_sum_years["Count"]), trendline="ols")
fig2 = px.scatter(x=df_sum_years.index, y=list(df_sum_years["Count"]), trendline="expanding", trendline_color_override="red")
fig3 = px.scatter(x=df_sum_years.index, y=list(df_sum_years["Count"]), trendline="ewm", trendline_options=dict(halflife=4), trendline_color_override="purple")
# fig4 = px.scatter(x=df_sum_years.index, y=list(df_sum_years["Count"]), trendline="rolling", trendline_options=dict(window=5), trendline_color_override="green")

trace1 = fig1['data']
trace2 = fig2['data']
trace3 = fig3['data']
# trace4 = fig4['data']

fig = make_subplots(rows=1, cols=1, shared_xaxes=False)
fig.add_trace(trace1[0], row=1, col=1)
fig.add_trace(trace1[1], row=1, col=1)
fig.add_trace(trace2[1], row=1, col=1)
fig.add_trace(trace3[1], row=1, col=1)
# fig.add_trace(trace4[1], row=1, col=1)
fig.update_layout(height=600, width=1000, title_text="Green Building Projects by Year", showlegend=False, xaxis_title="Year", yaxis_title="Count")
fig.show()

In [ ]:
X = df_sum_years.index.values.reshape(-1,1)
y = df_sum_years["Count"].values.reshape(-1,1)
eval_linear_reg(X, y)

-----------------------------------------------
The linear model is: Y = -8926.1 + 4.4571X
-----------------------------------------------
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.152
Model:                            OLS   Adj. R-squared:                  0.087
Method:                 Least Squares   F-statistic:                     2.339
Date:                Mon, 28 Feb 2022   Prob (F-statistic):              0.150
Time:                        20:06:24   Log-Likelihood:                -78.517
No. Observations:                  15   AIC:                             161.0
Df Residuals:                      13   BIC:                             162.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025     

In [ ]:
fig1 = px.scatter(x=df_sum_years_no_2022.index, y=list(df_sum_years_no_2022["Count"]), trendline="ols")
fig2 = px.scatter(x=df_sum_years_no_2022.index, y=list(df_sum_years_no_2022["Count"]), trendline="expanding", trendline_color_override="red")
fig3 = px.scatter(x=df_sum_years_no_2022.index, y=list(df_sum_years_no_2022["Count"]), trendline="ewm", trendline_options=dict(halflife=4), trendline_color_override="purple")
# fig4 = px.scatter(x=df_sum_years_no_2022.index, y=list(df_sum_years_no_2022["Count"]), trendline="rolling", trendline_options=dict(window=5), trendline_color_override="green")

trace1 = fig1['data']
trace2 = fig2['data']
trace3 = fig3['data']
# trace4 = fig4['data']

fig = make_subplots(rows=1, cols=1, shared_xaxes=False)
fig.add_trace(trace1[0], row=1, col=1)
fig.add_trace(trace1[1], row=1, col=1)
fig.add_trace(trace2[1], row=1, col=1)
fig.add_trace(trace3[1], row=1, col=1)
# fig.add_trace(trace4[1], row=1, col=1)
fig.update_layout(height=600, width=1000, title_text="Green Building Projects by Year - Without 2022", showlegend=False, xaxis_title="Year", yaxis_title="Count")
fig.show()

In [ ]:
X = df_sum_years_no_2022.index.values.reshape(-1,1)
y = df_sum_years_no_2022["Count"].values.reshape(-1,1)
eval_linear_reg(X, y)

-----------------------------------------------
The linear model is: Y = -1.4317e+04 + 7.1363X
-----------------------------------------------
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.343
Model:                            OLS   Adj. R-squared:                  0.288
Method:                 Least Squares   F-statistic:                     6.260
Date:                Mon, 28 Feb 2022   Prob (F-statistic):             0.0278
Time:                        20:06:24   Log-Likelihood:                -71.450
No. Observations:                  14   AIC:                             146.9
Df Residuals:                      12   BIC:                             148.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025 

####Area

In [ ]:
fig1 = px.scatter(x=df_sum_years.index, y=list(df_sum_years["Square Meter"]), trendline="ols")
fig2 = px.scatter(x=df_sum_years.index, y=list(df_sum_years["Square Meter"]), trendline="expanding", trendline_color_override="red")
fig3 = px.scatter(x=df_sum_years.index, y=list(df_sum_years["Square Meter"]), trendline="ewm", trendline_options=dict(halflife=4), trendline_color_override="purple")

trace1 = fig1['data']
trace2 = fig2['data']
trace3 = fig3['data']
# trace4 = fig4['data']

fig = make_subplots(rows=1, cols=1, shared_xaxes=False)
fig.add_trace(trace1[0], row=1, col=1)
fig.add_trace(trace1[1], row=1, col=1)
fig.add_trace(trace2[1], row=1, col=1)
fig.add_trace(trace3[1], row=1, col=1)
# fig.add_trace(trace4[1], row=1, col=1)
fig.update_layout(height=600, width=1000, title_text="Green Building Projects' Area by Year", showlegend=False, xaxis_title="Year", yaxis_title="Area (square meter)")
fig.show()

In [ ]:
X = df_sum_years.index.values.reshape(-1,1)
y = df_sum_years["Square Meter"].values.reshape(-1,1)
eval_linear_reg(X, y)

-----------------------------------------------
The linear model is: Y = -5.7321e+07 + 2.8627e+04X
-----------------------------------------------
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.174
Model:                            OLS   Adj. R-squared:                  0.111
Method:                 Least Squares   F-statistic:                     2.743
Date:                Mon, 28 Feb 2022   Prob (F-statistic):              0.122
Time:                        20:06:25   Log-Likelihood:                -208.84
No. Observations:                  15   AIC:                             421.7
Df Residuals:                      13   BIC:                             423.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.

In [ ]:
fig1 = px.scatter(x=df_sum_years_no_2022.index, y=list(df_sum_years_no_2022["Square Meter"]), trendline="ols")
fig2 = px.scatter(x=df_sum_years_no_2022.index, y=list(df_sum_years_no_2022["Square Meter"]), trendline="expanding", trendline_color_override="red")
fig3 = px.scatter(x=df_sum_years_no_2022.index, y=list(df_sum_years_no_2022["Square Meter"]), trendline="ewm", trendline_options=dict(halflife=4), trendline_color_override="purple")

trace1 = fig1['data']
trace2 = fig2['data']
trace3 = fig3['data']
# trace4 = fig4['data']

fig = make_subplots(rows=1, cols=1, shared_xaxes=False)
fig.add_trace(trace1[0], row=1, col=1)
fig.add_trace(trace1[1], row=1, col=1)
fig.add_trace(trace2[1], row=1, col=1)
fig.add_trace(trace3[1], row=1, col=1)
# fig.add_trace(trace4[1], row=1, col=1)
fig.update_layout(height=600, width=1000, title_text="Green Building Projects' Area by Year - Without 2022", showlegend=False, xaxis_title="Year", yaxis_title="Area (square meter)")
fig.show()

In [ ]:
X = df_sum_years_no_2022.index.values.reshape(-1,1)
y = df_sum_years_no_2022["Square Meter"].values.reshape(-1,1)
eval_linear_reg(X, y)

-----------------------------------------------
The linear model is: Y = -9.1979e+07 + 4.585e+04X
-----------------------------------------------
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.397
Model:                            OLS   Adj. R-squared:                  0.346
Method:                 Least Squares   F-statistic:                     7.887
Date:                Mon, 28 Feb 2022   Prob (F-statistic):             0.0158
Time:                        20:06:25   Log-Likelihood:                -192.58
No. Observations:                  14   AIC:                             389.2
Df Residuals:                      12   BIC:                             390.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.0

####Floors

In [ ]:
fig1 = px.scatter(x=df_sum_years.index, y=list(df_sum_years["Floors"]), trendline="ols")
fig2 = px.scatter(x=df_sum_years.index, y=list(df_sum_years["Floors"]), trendline="expanding", trendline_color_override="red")
fig3 = px.scatter(x=df_sum_years.index, y=list(df_sum_years["Floors"]), trendline="ewm", trendline_options=dict(halflife=4), trendline_color_override="purple")

trace1 = fig1['data']
trace2 = fig2['data']
trace3 = fig3['data']
# trace4 = fig4['data']

fig = make_subplots(rows=1, cols=1, shared_xaxes=False)
fig.add_trace(trace1[0], row=1, col=1)
fig.add_trace(trace1[1], row=1, col=1)
fig.add_trace(trace2[1], row=1, col=1)
fig.add_trace(trace3[1], row=1, col=1)
# fig.add_trace(trace4[1], row=1, col=1)
fig.update_layout(height=600, width=1000, title_text="Green Building Projects' Floors by Year", showlegend=False, xaxis_title="Year", yaxis_title="Floors")
fig.show()

In [ ]:
X = df_sum_years.index.values.reshape(-1,1)
y = df_sum_years["Floors"].values.reshape(-1,1)
eval_linear_reg(X, y)

-----------------------------------------------
The linear model is: Y = -7.8231e+04 + 39.075X
-----------------------------------------------
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.145
Model:                            OLS   Adj. R-squared:                  0.080
Method:                 Least Squares   F-statistic:                     2.211
Date:                Mon, 28 Feb 2022   Prob (F-statistic):              0.161
Time:                        20:06:25   Log-Likelihood:                -111.50
No. Observations:                  15   AIC:                             227.0
Df Residuals:                      13   BIC:                             228.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025 

In [ ]:
fig1 = px.scatter(x=df_sum_years_no_2022.index, y=list(df_sum_years_no_2022["Floors"]), trendline="ols")
fig2 = px.scatter(x=df_sum_years_no_2022.index, y=list(df_sum_years_no_2022["Floors"]), trendline="expanding", trendline_color_override="red")
fig3 = px.scatter(x=df_sum_years_no_2022.index, y=list(df_sum_years_no_2022["Floors"]), trendline="ewm", trendline_options=dict(halflife=4), trendline_color_override="purple")

trace1 = fig1['data']
trace2 = fig2['data']
trace3 = fig3['data']
# trace4 = fig4['data']

fig = make_subplots(rows=1, cols=1, shared_xaxes=False)
fig.add_trace(trace1[0], row=1, col=1)
fig.add_trace(trace1[1], row=1, col=1)
fig.add_trace(trace2[1], row=1, col=1)
fig.add_trace(trace3[1], row=1, col=1)
# fig.add_trace(trace4[1], row=1, col=1)
fig.update_layout(height=600, width=1000, title_text="Green Building Projects' Floors by Year - Without 2022", showlegend=False, xaxis_title="Year", yaxis_title="Floors")
fig.show()

In [ ]:
X = df_sum_years_no_2022.index.values.reshape(-1,1)
y = df_sum_years_no_2022["Floors"].values.reshape(-1,1)
eval_linear_reg(X, y)

-----------------------------------------------
The linear model is: Y = -1.2828e+05 + 63.946X
-----------------------------------------------
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.345
Model:                            OLS   Adj. R-squared:                  0.291
Method:                 Least Squares   F-statistic:                     6.332
Date:                Mon, 28 Feb 2022   Prob (F-statistic):             0.0271
Time:                        20:06:25   Log-Likelihood:                -102.07
No. Observations:                  14   AIC:                             208.1
Df Residuals:                      12   BIC:                             209.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025 

####Housing Units

In [ ]:
fig1 = px.scatter(x=df_sum_years.index, y=list(df_sum_years["Housing Units"]), trendline="ols")
fig2 = px.scatter(x=df_sum_years.index, y=list(df_sum_years["Housing Units"]), trendline="expanding", trendline_color_override="red")
fig3 = px.scatter(x=df_sum_years.index, y=list(df_sum_years["Housing Units"]), trendline="ewm", trendline_options=dict(halflife=4), trendline_color_override="purple")

trace1 = fig1['data']
trace2 = fig2['data']
trace3 = fig3['data']
# trace4 = fig4['data']

fig = make_subplots(rows=1, cols=1, shared_xaxes=False)
fig.add_trace(trace1[0], row=1, col=1)
fig.add_trace(trace1[1], row=1, col=1)
fig.add_trace(trace2[1], row=1, col=1)
fig.add_trace(trace3[1], row=1, col=1)
# fig.add_trace(trace4[1], row=1, col=1)
fig.update_layout(height=600, width=1000, title_text="Green Building Projects' Housing Units by Year", showlegend=False, xaxis_title="Year", yaxis_title="Housing Units")
fig.show()

In [ ]:
X = df_sum_years.index.values.reshape(-1,1)
y = df_sum_years["Housing Units"].values.reshape(-1,1)
eval_linear_reg(X, y)

-----------------------------------------------
The linear model is: Y = -3.3371e+05 + 166.52X
-----------------------------------------------
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.218
Model:                            OLS   Adj. R-squared:                  0.157
Method:                 Least Squares   F-statistic:                     3.613
Date:                Mon, 28 Feb 2022   Prob (F-statistic):             0.0797
Time:                        20:06:25   Log-Likelihood:                -129.56
No. Observations:                  15   AIC:                             263.1
Df Residuals:                      13   BIC:                             264.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025 

In [ ]:
fig1 = px.scatter(x=df_sum_years_no_2022.index, y=list(df_sum_years_no_2022["Housing Units"]), trendline="ols")
fig2 = px.scatter(x=df_sum_years_no_2022.index, y=list(df_sum_years_no_2022["Housing Units"]), trendline="expanding", trendline_color_override="red")
fig3 = px.scatter(x=df_sum_years_no_2022.index, y=list(df_sum_years_no_2022["Housing Units"]), trendline="ewm", trendline_options=dict(halflife=4), trendline_color_override="purple")

trace1 = fig1['data']
trace2 = fig2['data']
trace3 = fig3['data']
# trace4 = fig4['data']

fig = make_subplots(rows=1, cols=1, shared_xaxes=False)
fig.add_trace(trace1[0], row=1, col=1)
fig.add_trace(trace1[1], row=1, col=1)
fig.add_trace(trace2[1], row=1, col=1)
fig.add_trace(trace3[1], row=1, col=1)
# fig.add_trace(trace4[1], row=1, col=1)
fig.update_layout(height=600, width=1000, title_text="Green Building Projects' Housing Units by Year - Without 2022", showlegend=False, xaxis_title="Year", yaxis_title="Housing Units")
fig.show()

In [ ]:
X = df_sum_years_no_2022.index.values.reshape(-1,1)
y = df_sum_years_no_2022["Housing Units"].values.reshape(-1,1)
eval_linear_reg(X, y)

-----------------------------------------------
The linear model is: Y = -5.2393e+05 + 261.05X
-----------------------------------------------
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.476
Model:                            OLS   Adj. R-squared:                  0.432
Method:                 Least Squares   F-statistic:                     10.89
Date:                Mon, 28 Feb 2022   Prob (F-statistic):            0.00635
Time:                        20:06:26   Log-Likelihood:                -117.97
No. Observations:                  14   AIC:                             239.9
Df Residuals:                      12   BIC:                             241.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025 

##By Municipality

In [ ]:

# # group_years.cumcount().to_frame(name="Unique Years")
# def f(group):
#   local_auth = group["Local Authority"]
#   local_auth_years_counts = dict(Counter(list(group['Year'])))
#   print(local_auth_years_counts)
#   local_auth_years_counts = {str(key): value for key, value in local_auth_years_counts.items()}
#   print(local_auth_years_counts)

#   print(pd.DataFrame(local_auth_years_counts, index=[local_auth]))

# group_years = df_combined_green_building[["Local Authority", "Year"]].groupby(["Local Authority"])
# groups_years = group_years.apply(lambda x: f(x))#len(x['Year']))
# groups_years
#Count how many new projects in each year for each local authority 

In [ ]:
df_green_building_value_counts = df_combined_green_building["Local Authority"].value_counts().sort_index()
df_green_building_value_counts = df_green_building_value_counts.to_frame(name="Count")

group_housing = df_combined_green_building[["Local Authority","Housing Units"]].groupby("Local Authority")
group_housing_sum = group_housing.sum()

group_floors = df_combined_green_building[["Local Authority", "Floors"]].groupby("Local Authority")
group_floors_sum = group_floors.sum()

group_area = df_combined_green_building[["Local Authority", "Square Meter"]].groupby("Local Authority")
group_area_sum = group_area.sum()

group_years = df_combined_green_building[["Local Authority", "Year"]].groupby("Local Authority")
group_years = group_years.apply(lambda x: x['Year'].unique())
group_years = group_years.to_frame(name="Unique Years")
group_years["Unique Years Count"] = group_years["Unique Years"].apply(lambda r: len(r))

df_sum_local_auth = pd.concat([df_green_building_value_counts,group_floors_sum, group_housing_sum,group_area_sum, group_years], axis=1, join="outer")

df_sum_local_auth["Housing Units Per Project"] = df_sum_local_auth["Housing Units"]/df_sum_local_auth["Count"]
df_sum_local_auth["Floors Per Project"] = df_sum_local_auth["Floors"]/df_sum_local_auth["Count"]
df_sum_local_auth["Square Meter Per Project"] = df_sum_local_auth["Square Meter"]/df_sum_local_auth["Count"]
df_sum_local_auth = df_sum_local_auth[["Count","Housing Units","Housing Units Per Project","Floors","Floors Per Project", "Square Meter", "Square Meter Per Project", "Unique Years", "Unique Years Count"]]
df_sum_local_auth

,Count,Housing Units,Housing Units Per Project,Floors,Floors Per Project,Square Meter,Square Meter Per Project,Unique Years,Unique Years Count
אום אל-פחם,1,0,0.000000,5.0,5.000000,3500.00,3500.000000,[2021],1
אופקים,1,0,0.000000,2.0,2.000000,4090.00,4090.000000,[2021],1
אור עקיבא,4,248,62.000000,39.0,9.750000,17388.00,4347.000000,"[2019, 2018]",2
אריאל,10,160,16.000000,50.0,5.000000,13150.00,1315.000000,"[2018, 2014]",2
אשדוד,10,600,60.000000,85.0,8.500000,196782.00,19678.200000,"[2017, 2020, 2013, 2021]",4
אשקלון,39,1034,26.512821,366.0,9.384615,143498.00,3679.435897,"[2013, 2014, 2015, 2016, 2017, 2019, 2021]",7
באר יעקב,15,800,53.333333,219.0,14.600000,94057.00,6270.466667,"[2013, 2014, 2018, 2019, 2017, 2016, 2011, 2021]",8
באר שבע,6,169,28.166667,40.0,6.666667,67753.00,11292.166667,"[2008, 2019, 2017, 2021]",4
בית שמש,3,917,305.666667,67.0,22.333333,148000.00,49333.333333,"[2016, 2021]",2
בני ברק,1,0,0.000000,17.0,17.000000,28521.00,28521.000000,[2021],1


###Statistics

In [ ]:
columns = ['Count', 'Housing Units', 'Housing Units Per Project', 'Floors',
       'Floors Per Project', 'Square Meter', 'Square Meter Per Project', 'Unique Years Count']
summary_df = get_summary(df_sum_local_auth, columns)
print("Municipality Statistics")
summary_df

Municipality Statistics


,Mean,Stdev,Median,Min,Max
Count,13.767,23.234,7.000,1.0,133.000
Housing Units,455.433,658.326,210.000,0.0,3471.000
Housing Units Per Project,30.100,43.125,25.733,0.0,305.667
Floors,126.225,193.388,60.500,1.0,967.000
Floors Per Project,8.079,4.625,8.000,1.0,22.333
Square Meter,90940.211,112859.287,49399.000,700.0,517402.000
Square Meter Per Project,9067.976,10841.875,4967.800,700.0,55814.500
Unique Years Count,3.700,2.800,3.000,1.0,12.000


###Diffrences Visualiztion

In [ ]:
def years_to_str(r):
  string = str(r[0])
  for year in r:
    string += ", " 
    string += str(year)
  return string
df_sum_local_auth["Unique Years String"] = df_sum_local_auth["Unique Years"].apply(lambda r: years_to_str(r))


In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

fig = make_subplots(rows=5, cols=2, subplot_titles=["Project Count","Unique Years Count","Total Square Meter", "Square Meter Per Project","Total Floors", "Floors Per Project","Total Housing Units", "Housing Units Per Project"],
                    column_widths=[0.5, 0.5],vertical_spacing=0.09, horizontal_spacing=0.05)

k=50

fig.add_trace(
    go.Bar(x=df_sum_local_auth.sort_values("Count").index, y=sorted(list(df_sum_local_auth["Count"]))),
    row=1, col=1)

fig.add_trace(
    go.Bar(x=df_sum_local_auth.sort_values("Unique Years Count").index, y=sorted(list(df_sum_local_auth["Unique Years Count"]))
    , hovertext=sorted(list(df_sum_local_auth["Unique Years String"]))),
    row=1, col=2)

fig.add_trace(
    go.Bar(x=df_sum_local_auth.sort_values("Square Meter").index, y=sorted(list(df_sum_local_auth["Square Meter"]))),
    row=2, col=1)

fig.add_trace(
    go.Bar(x=df_sum_local_auth.sort_values("Square Meter Per Project").index, y=sorted(list(df_sum_local_auth["Square Meter Per Project"]))),
    row=2, col=2)

fig.add_trace(
    go.Bar(x=df_sum_local_auth.sort_values("Floors").index, y=sorted(list(df_sum_local_auth["Floors"]))),
    row=3, col=1)

fig.add_trace(
    go.Bar(x=df_sum_local_auth.sort_values("Floors Per Project").index, y=sorted(list(df_sum_local_auth["Floors Per Project"]))),
    row=3, col=2)

fig.add_trace(
    go.Bar(x=df_sum_local_auth.sort_values("Housing Units").index, y=sorted(list(df_sum_local_auth["Housing Units"]))),
    row=4, col=1)

fig.add_trace(
    go.Bar(x=df_sum_local_auth.sort_values("Housing Units Per Project").index, y=sorted(list(df_sum_local_auth["Housing Units Per Project"]))),
    row=4, col=2)

fig.update_layout(height=1200, width=1000, title_text="Changes by Municipality", showlegend=False)
fig.show()

#

In [ ]:
from collections import Counter

unique_years = df_combined_green_building["Year"].unique()
unique_local_auth = df_combined_green_building["Local Authority"].unique()


local_auth_years_counts_df = pd.DataFrame(columns=unique_years, index=unique_local_auth)
local_auth_years_counts_df = local_auth_years_counts_df.fillna(0)

for local_auth in unique_local_auth:
  local_auth_df = df_combined_green_building[df_combined_green_building["Local Authority"]==local_auth]
  local_auth_counter = Counter(local_auth_df["Year"])
  for year, count in local_auth_counter.items():
    local_auth_years_counts_df.at[local_auth, year] = count
    
local_auth_years_counts_df = local_auth_years_counts_df.sort_index()
local_auth_years_counts_df = local_auth_years_counts_df.reindex(sorted(local_auth_years_counts_df.columns), axis=1)
local_auth_years_counts_df

,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
אום אל-פחם,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
אופקים,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
אור עקיבא,0,0,0,0,0,0,0,0,0,0,2,2,0,0,0
אריאל,0,0,0,0,0,0,5,0,0,0,5,0,0,0,0
אשדוד,0,0,0,0,0,1,0,0,0,1,0,0,5,3,0
אשקלון,0,0,0,0,0,1,18,3,4,7,0,2,0,4,0
באר יעקב,0,0,0,1,0,2,2,0,2,2,2,3,0,1,0
באר שבע,1,0,0,0,0,0,0,0,0,1,0,1,0,3,0
בית שמש,0,0,0,0,0,0,0,0,2,0,0,0,0,1,0
בני ברק,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [ ]:
local_auth_years_counts_dict = {"Year":[], "Count": [], "Local Authority": []}
for index,row in local_auth_years_counts_df.iterrows():
  # print(index)
  for col in row.index:
    local_auth_years_counts_dict["Year"].append(col)
    local_auth_years_counts_dict["Count"].append(row.loc[col])
    local_auth_years_counts_dict["Local Authority"].append(index)
local_auth_years_counts_df_cols = pd.DataFrame(local_auth_years_counts_dict)
local_auth_years_counts_df_cols

,Year,Count,Local Authority
0,2008,0,אום אל-פחם
1,2009,0,אום אל-פחם
2,2010,0,אום אל-פחם
3,2011,0,אום אל-פחם
4,2012,0,אום אל-פחם
...,...,...,...
895,2018,0,תמר
896,2019,0,תמר
897,2020,0,תמר
898,2021,0,תמר


In [ ]:
local_auth_years_cumulative_counts_df = local_auth_years_counts_df.copy(deep=True)
local_auth_years_cumulative_counts_df = local_auth_years_cumulative_counts_df.cumsum(axis=1)
local_auth_years_cumulative_counts_df = local_auth_years_cumulative_counts_df.sort_values(by=2022)
local_auth_years_cumulative_counts_df

,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
אום אל-פחם,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
מזכרת בתיה,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1
מטה יהודה,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1
מעלה יוסף,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1
דימונה,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
משגב,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1
נהרייה,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1
בני שמעון,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
זכרון יעקב,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
בני ברק,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1


In [ ]:
local_auth_years_cumulative_counts_dict = {"Year":[], "Cumulative Count": [], "Local Authority": []}
for index,row in local_auth_years_cumulative_counts_df.iterrows():
  # print(index)
  for col in row.index:
    local_auth_years_cumulative_counts_dict["Year"].append(col)
    local_auth_years_cumulative_counts_dict["Cumulative Count"].append(row.loc[col])
    local_auth_years_cumulative_counts_dict["Local Authority"].append(index)
local_auth_years_cumulative_counts_df_cols = pd.DataFrame(local_auth_years_cumulative_counts_dict)
local_auth_years_cumulative_counts_df_cols

,Year,Cumulative Count,Local Authority
0,2008,0,אום אל-פחם
1,2009,0,אום אל-פחם
2,2010,0,אום אל-פחם
3,2011,0,אום אל-פחם
4,2012,0,אום אל-פחם
...,...,...,...
895,2018,114,רעננה
896,2019,117,רעננה
897,2020,118,רעננה
898,2021,132,רעננה


In [ ]:
fig = px.line(local_auth_years_counts_df_cols, x="Year", y="Count", color="Local Authority")
fig.show()

In [ ]:
fig = px.area(local_auth_years_cumulative_counts_df_cols, x="Year", y="Cumulative Count", color="Local Authority", width=1000)
fig.show()

##Additional Metrics

In [ ]:
df_green_building_addit = df_green_building[["Local Authority", "Year", "Final Score", "Rank", "New/Renovation"]]
df_green_building_addit


,Local Authority,Year,Final Score,Rank,New/Renovation
0,באר שבע,2008,69,ירוק,שיפוץ
1,זכרון יעקב,2008,77.6,מצטיין,חדש
2,תל אביב-יפו,2009,NaN,ירוק,חדש
3,חיפה,2010,77.3,מצטיין,חדש
4,יקנעם עילית,2010,65,ירוק,חדש
...,...,...,...,...,...
444,רעננה,2019,56,1 כוכב,חדש
445,רעננה,2019,56,1 כוכב,חדש
446,נס ציונה,2019,56,1 כוכב,חדש
447,נס ציונה,2019,56,1 כוכב,חדש


In [ ]:
df_active_green_building_addit = df_active_green_building[["Local Authority", "Year", "New/Renovation"]]
df_active_green_building_addit

,Local Authority,Year,New/Renovation
0,יבנה,2022,חדש
1,דימונה,2021,חדש
2,תל אביב-יפו,2020,חדש
3,אשדוד,2020,חדש
4,תל אביב-יפו,2019,חדש
...,...,...,...
372,תל אביב-יפו,2021,חדש
373,רמת גן,2021,חדש
374,חדרה,2021,חדש
375,רעננה,2022,חדש


###Final Score

In [ ]:
df_score = df_green_building_addit.copy(deep=True)
df_score["Final Score"] = df_score["Final Score"].apply(lambda r: r if type(r) != str else None)
df_score = df_score.dropna()


In [ ]:
columns = ["Final Score"]
summary_df = get_summary(df_score, columns)
print("Final Score Statistics")
summary_df

Final Score Statistics


,Max,Mean,Median,Min,Stdev
Final Score,90.0,60.154,59.0,43.0,5.458


In [ ]:
score_group = df_score[["Year","Final Score"]].groupby("Year")
score_group.mean()

,Final Score
Year,
2008,73.300000
2010,69.450000
2011,60.000000
2012,62.677778
2013,62.436757
2014,60.595922
2015,60.971458
2016,59.704545
2017,59.558824


In [ ]:
fig = px.scatter(df_score, x="Year", y="Final Score", color="Local Authority")
fig.show()

In [ ]:
fig = px.violin(df_score, x="Year", y="Final Score", box=True, )#, color="Local Authority")
fig.show()

In [ ]:
fig = px.violin(df_score, y="Final Score", points="all", box=True, width =800)#, color="Local Authority")
fig.show()

###Rank

In [ ]:
# d = {
#   "כוכב 1": {"min": 43, "max":64},
#   "כוכבים 2": {"min": 50, "max": 71}, 
#   "כוכבים 3": {"min": 62, "max": 77}, 
#   "כוכבים 4": {"min": 70.5, "max": 73.5}, 
#   "כוכבים 5": {"min": 90, "max": 90}, 
#   "ירוק": {"min": 55, "max": 74.5}, 
#   "ירוק מצטיין": {"min": 76, "max": 76}, 
#   "מצטיין": {"min": 75, "max": 80.3}    
# }

In [ ]:
df_rank = df_green_building_addit.copy(deep=True)
print(len(df_rank))
# df_score["Rank"] = df_score["Rank"].apply(lambda r: r if type(r) != str else None)
df_rank = df_rank.dropna()
print(len(df_rank))
# df_rank["Rank"]

449
443


In [ ]:
print("Rank Statistics")
df_rank["Rank"].describe()

Rank Statistics


count        443
unique         8
top       1 כוכב
freq         215
Name: Rank, dtype: object

In [ ]:
rank_group = df_score[["Year","Rank"]].groupby("Year")
rank_group.describe()

Rank                    
     count unique     top freq
Year                          
2008     2      2    ירוק    1
2010     4      2    ירוק    3
2011     2      1    ירוק    2
2012    27      2    ירוק   22
2013    37      4    ירוק   32
2014   103      4    ירוק   88
2015    48      6    ירוק   24
2016    44      2  1 כוכב   41
2017    68      5  1 כוכב   57
2018    70      4  1 כוכב   59
2019    36      2  1 כוכב   34

In [ ]:
fig = px.histogram(df_rank, x="Rank", width=600, height=400)#, animation_frame="Year")#), color="Local Authority")
fig.show()

In [ ]:
fig = px.pie(df_rank, names="Rank", width=500, height=500)#, color_discrete_sequence=px.colors.sequential.RdBu)
fig.show()

In [ ]:
# fig = px.violin(df_rank, x="Year", y="Rank")#, color="Local Authority")
# fig.show()

In [ ]:
fig = px.violin(df_rank, y="Rank", points="all", box=True)#, color="Local Authority")
fig.show()

###New/Renovation

In [ ]:
df_type = df_green_building_addit.copy(deep=True)[["Local Authority", "Year", "New/Renovation"]]
#Missing data
df_type["New/Renovation"] = df_type["New/Renovation"].apply(lambda r: r if r != "nan" else None)
df_type = df_type.dropna()
#preprocess
df_type["New/Renovation"] = df_type["New/Renovation"].apply(lambda r: r if r != "קיים" else "שיפוץ")

df_active_green_building_addit["New/Renovation"] = df_active_green_building_addit["New/Renovation"].apply(lambda r: r if r != "תוספת" else "שיפוץ")

df_type_merged = pd.concat([df_type,df_active_green_building_addit])
df_type_merged

,Local Authority,Year,New/Renovation
0,באר שבע,2008,שיפוץ
1,זכרון יעקב,2008,חדש
2,תל אביב-יפו,2009,חדש
3,חיפה,2010,חדש
4,יקנעם עילית,2010,חדש
...,...,...,...
372,תל אביב-יפו,2021,חדש
373,רמת גן,2021,חדש
374,חדרה,2021,חדש
375,רעננה,2022,חדש


In [ ]:
print("New/Renovation Statistics")
print(df_type_merged["New/Renovation"].describe())
print(df_type_merged["New/Renovation"].value_counts())

New/Renovation Statistics
count     823
unique      2
top       חדש
freq      803
Name: New/Renovation, dtype: object
חדש      803
שיפוץ     20
Name: New/Renovation, dtype: int64


In [ ]:
type_group = df_type_merged[["Year","New/Renovation"]].groupby("Year")
type_group.describe()

New/Renovation                   
              count unique    top freq
Year                                  
2008              2      2  שיפוץ    1
2009              1      1    חדש    1
2010              3      1    חדש    3
2011              3      1    חדש    3
2012             28      1    חדש   28
2013             75      1    חדש   75
2014            154      2    חדש  153
2015             93      2    חדש   89
2016             79      2    חדש   77
2017             92      2    חדש   91
2018             76      2    חדש   75
2019             38      1    חדש   38
2020             33      1    חדש   33
2021            141      2    חדש  131
2022              5      1    חדש    5

In [ ]:
fig = px.violin(df_type_merged, x="Year", y="New/Renovation")#, color="Local Authority")
fig.show()

In [ ]:
fig = px.histogram(df_type_merged, x="New/Renovation", width=400, height=400)#, points="all", box=True)#, color="Local Authority")
fig.show()

In [ ]:
fig = px.pie(df_type_merged, names="New/Renovation", width=500, height=500)#, color_discrete_sequence=px.colors.sequential.RdBu)
fig.show()

##Municipalities Analysis

In [ ]:
df_municipalities_combined_all = pd.merge(df_combined_green_building, df_municipalities, left_on="Local Authority", right_on="Local Authority",  how="left")
df_municipalities_combined_all = df_municipalities_combined_all[df_municipalities_combined_all["Local Authority"] != "רשות שדות התעופה"]
# df_municipalities_combined_all = df_municipalities_combined_all.drop("שם הרשות", axis=1)
df_municipalities_combined_all

,Id,Designation,Local Authority,Street,Housing Units,Floors,Square Meter,Year,lat,long,New/Renovation,Year Date,Distance from Tel-Aviv District Border Line (km),Area (square km),Population,Avrege Daily Unemployment Benefit,Average Monthly Wage of Employees,Number of Employees,Percentage of Employees Earning Up To Minimum Wage,Number of Self-Employed,Average Monthly Wage of Self-Employed,Percentage of Self-Employed Earning Up To Half the Average Wage,"Employees Inequality Index (Gini Index, 0 is complete equality)",Percentage of students dropping out,Percentage of matriculation certificate eligible among 12th graders,Percentage of students out of the general population,Population density per square kilometer
0,1,Offices,באר שבע,מנחם קרויצר,0,8.0,5865.0,2008,31.246124,34.802489,NaN,2008-01-01,90.0,117.398456,210595.0,167.262,9546.313200,109665.0,38.1,6013.0,10497.269330,41.8,0.421392,0.896376,78.332645,3.89,1793.8
1,2,Institution,זכרון יעקב,גני רמת הנדיב,0,2.0,2800.0,2008,32.553139,34.945037,NaN,2008-01-01,48.3,31.527262,23437.0,188.032,13301.357820,10388.0,36.6,1287.0,11749.285140,40.2,0.522885,1.249024,81.250000,4.17,743.4
2,3,Institution,תל אביב-יפו,הארבעה,0,1.0,551.0,2009,32.070562,34.786871,NaN,2009-01-01,0.0,51.788550,463808.0,210.111,12590.862590,236408.0,31.4,32674.0,12756.820220,36.7,0.470783,1.104951,78.391044,5.10,8955.8
3,4,Offices,חיפה,אנדריי סחרוב,0,7.0,50000.0,2010,32.790836,34.961088,NaN,2010-01-01,77.5,64.560524,283736.0,172.116,10798.161010,137442.0,37.7,11621.0,11029.107540,42.2,0.473584,0.969912,75.432756,4.91,4394.9
4,5,Institution,יקנעם עילית,שדרות רבין,0,8.0,3961.0,2010,32.643746,35.092642,NaN,2010-01-01,65.9,8.311365,24091.0,185.923,11690.847340,11956.0,32.0,699.0,11418.719070,36.1,0.430354,0.421408,83.653846,3.28,2898.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
821,754,Residence,תל אביב-יפו,בן יהודה,14,7.0,0.0,2021,32.075191,34.767907,חדש,2021-01-01,0.0,51.788550,463808.0,210.111,12590.862590,236408.0,31.4,32674.0,12756.820220,36.7,0.470783,1.104951,78.391044,5.10,8955.8
822,757,Residence,רמת גן,צאלים,73,7.0,5000.0,2021,32.066350,34.822675,חדש,2021-01-01,0.0,16.394978,167556.0,200.817,12339.145710,78651.0,30.2,9427.0,12657.212040,36.4,0.445084,0.650894,88.367347,4.48,10220.0
823,773,Residence,חדרה,אחד העם,12,7.0,448.0,2021,32.433180,34.923864,חדש,2021-01-01,32.7,51.027405,98908.0,181.014,9439.260281,48802.0,37.1,4157.0,9712.545853,41.8,0.407895,0.689381,74.764756,2.32,1938.3
824,777,Public structure,רעננה,אלפסי,0,2.0,530.0,2022,32.190872,34.872009,חדש,2022-01-01,2.3,14.854404,76277.0,200.301,12925.757600,36620.0,34.2,4786.0,13019.419160,36.6,0.491900,0.746062,85.135135,3.88,5135.0


In [ ]:
df_sum_local_auth_temp = df_sum_local_auth.copy(deep=True)
df_sum_local_auth_temp["Local Authority"] = df_sum_local_auth_temp.index
df_sum_local_auth_temp = df_sum_local_auth_temp.reset_index(drop=True)

In [ ]:
df_municipalities_combined = pd.merge(df_sum_local_auth_temp, df_municipalities, left_on="Local Authority", right_on="Local Authority",  how="left")
df_municipalities_combined = df_municipalities_combined[df_municipalities_combined["Local Authority"] != "רשות שדות התעופה"]
# df_municipalities_combined = df_municipalities_combined.drop("שם הרשות", axis=1)
df_municipalities_combined

,Count,Housing Units,Housing Units Per Project,Floors,Floors Per Project,Square Meter,Square Meter Per Project,Unique Years,Unique Years Count,Unique Years String,Local Authority,Distance from Tel-Aviv District Border Line (km),Area (square km),Population,Avrege Daily Unemployment Benefit,Average Monthly Wage of Employees,Number of Employees,Percentage of Employees Earning Up To Minimum Wage,Number of Self-Employed,Average Monthly Wage of Self-Employed,Percentage of Self-Employed Earning Up To Half the Average Wage,"Employees Inequality Index (Gini Index, 0 is complete equality)",Percentage of students dropping out,Percentage of matriculation certificate eligible among 12th graders,Percentage of students out of the general population,Population density per square kilometer
0,1,0,0.000000,5.0,5.000000,3500.00,3500.000000,[2021],1,"2021, 2021",אום אל-פחם,57.8,25.956107,56777.0,152.414,5869.820349,22149.0,60.0,1918.0,8883.182912,40.4,0.384898,1.166139,63.313096,2.17,2187.4
1,1,0,0.000000,2.0,2.000000,4090.00,4090.000000,[2021],1,"2021, 2021",אופקים,86.8,16.352374,32555.0,161.041,7768.952111,14311.0,43.2,822.0,9870.667177,40.0,0.380631,1.059105,51.860465,1.96,1990.8
2,4,248,62.000000,39.0,9.750000,17388.00,4347.000000,"[2019, 2018]",2,"2019, 2019, 2018",אור עקיבא,39.5,5.546007,19447.0,179.951,8315.826367,9833.0,38.1,789.0,8808.060663,37.5,0.363485,1.011463,86.554622,2.12,3506.6
3,10,160,16.000000,50.0,5.000000,13150.00,1315.000000,"[2018, 2014]",2,"2018, 2018, 2014",אריאל,34.7,11.658523,19582.0,192.823,9670.024184,10596.0,32.7,657.0,9951.244216,41.6,0.383866,0.970149,88.265306,10.69,2358.2
4,10,600,60.000000,85.0,8.500000,196782.00,19678.200000,"[2017, 2020, 2013, 2021]",4,"2017, 2017, 2020, 2013, 2021",אשדוד,28.3,47.166811,226154.0,173.162,8922.497087,109476.0,40.1,7454.0,9738.221883,42.8,0.414305,0.740258,57.811586,2.27,4794.8
5,39,1034,26.512821,366.0,9.384615,143498.00,3679.435897,"[2013, 2014, 2015, 2016, 2017, 2019, 2021]",7,"2013, 2013, 2014, 2015, 2016, 2017, 2019, 2021",אשקלון,43.8,45.361012,146519.0,172.702,8847.256608,69453.0,39.5,5044.0,9777.040993,42.2,0.403477,1.079719,76.814988,2.59,3230.1
6,15,800,53.333333,219.0,14.600000,94057.00,6270.466667,"[2013, 2014, 2018, 2019, 2017, 2016, 2011, 2021]",8,"2013, 2013, 2014, 2018, 2019, 2017, 2016, 2011...",באר יעקב,9.5,9.462631,29015.0,210.852,12107.932180,10789.0,26.0,1032.0,12913.091650,29.0,0.382401,0.998004,72.796935,1.88,3066.2
7,6,169,28.166667,40.0,6.666667,67753.00,11292.166667,"[2008, 2019, 2017, 2021]",4,"2008, 2008, 2019, 2017, 2021",באר שבע,90.0,117.398456,210595.0,167.262,9546.313200,109665.0,38.1,6013.0,10497.269330,41.8,0.421392,0.896376,78.332645,3.89,1793.8
8,3,917,305.666667,67.0,22.333333,148000.00,49333.333333,"[2016, 2021]",2,"2016, 2016, 2021",בית שמש,41.9,38.289930,132544.0,158.591,7298.784512,38535.0,50.2,2954.0,8825.371444,46.9,0.435652,1.870951,33.943782,1.75,3461.6
9,1,0,0.000000,17.0,17.000000,28521.00,28521.000000,[2021],1,"2021, 2021",בני ברק,0.0,7.348485,208793.0,152.408,6468.876251,65137.0,51.9,4289.0,8596.710461,48.4,0.407546,1.733525,5.885904,0.85,28413.1


In [ ]:
def eval_linear_reg_multi_var(df, predictors, target):
  Xs = df.drop([target], axis=1)[predictors]
  y = df[target].values.reshape(-1,1)

  reg = LinearRegression()
  reg.fit(Xs, y)
  print("-----------------------------------------------")
  liner_model_str = "The linear model is: Y = {:.5} ".format(reg.intercept_[0])
  for index, predictor in enumerate(predictors):
    liner_model_str += "+ {:.5}*{}".format(reg.coef_[0][index], predictor)
  print(liner_model_str)
  print("-----------------------------------------------")

  X = np.column_stack([df[predictor] for predictor in predictors])
  y = df[target]
  X2 = sm.add_constant(X)
  est = sm.OLS(y, X2)
  est2 = est.fit()
  print(est2.summary())
  print("\npvalues\n", est2.pvalues)
  print("predictors")
  for index, predictor in enumerate(predictors):
    print(f"x{index+1}: {predictor}")

In [ ]:
columns = df_municipalities.columns.tolist()[1:]
columns_with_target = [elem for elem in columns]
columns_with_target = columns_with_target + ["Count"]
eval_linear_reg_multi_var(df_municipalities_combined[columns_with_target], predictors=columns, target="Count")


-----------------------------------------------
The linear model is: Y = 61.963 + -0.26963*Distance from Tel-Aviv District Border Line (km)+ 0.0011476*Area (square km)+ -3e-05*Population+ -0.32319*Avrege Daily Unemployment Benefit+ -0.01494*Average Monthly Wage of Employees+ -0.00019772*Number of Employees+ -2.8696*Percentage of Employees Earning Up To Minimum Wage+ 0.002916*Number of Self-Employed+ 0.0023842*Average Monthly Wage of Self-Employed+ -0.80469*Percentage of Self-Employed Earning Up To Half the Average Wage+ 528.72*Employees Inequality Index (Gini Index, 0 is complete equality)+ 19.711*Percentage of students dropping out+ 0.63274*Percentage of matriculation certificate eligible among 12th graders+ -1.2413*Percentage of students out of the general population+ 0.00014273*Population density per square kilometer
-----------------------------------------------
                            OLS Regression Results                            
Dep. Variable:                  Count   R